In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after

import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time


from datetime import datetime as dt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [11]:
os.environ['TwitterBearer'] = 'AAAAAAAAAAAAAAAAAAAAAOTlYgEAAAAAN6NQtvxLs8w6uYjCKHyus0P2qbQ%3DeZBReuwEU9IGT2DEbLrB0DYyd2xNc3oPND6amo5wGmTbKUTT1v'


In [12]:
def auth():
    return os.getenv('TwitterBearer')


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)


def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()




In [13]:
def fetch_tweets(keyword, total_tweets, max_results, start_list, end_list):
    # Create file
    csvFile = open(keyword+"_data.csv", "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Create headers for the data you want to save, in this example, we only want save these columns in our dataset
    csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
    csvFile.close()

    for i in range(0,len(start_list)):

        # Inputs
        count = 0 # Counting tweets per time period
        max_count = 100 # Max tweets per time period
        flag = True
        next_token = None
        
        # Check if flag is true
        while flag:
            # Check if max_count reached
            if count >= max_count:
                break
            print("-------------------")
            print("Token: ", next_token)
            url = create_url(keyword, start_list[i],end_list[i], max_results)
            json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
            result_count = json_response['meta']['result_count']

            if 'next_token' in json_response['meta']:
                # Save the token to use for next call
                next_token = json_response['meta']['next_token']
                print("Next Token: ", next_token)
                if result_count is not None and result_count > 0 and next_token is not None:
                    print("Start Date: ", start_list[i])
                    append_to_csv(json_response, keyword+"_data.csv")
                    count += result_count
                    total_tweets += result_count
                    print("Total # of Tweets added: ", total_tweets)
                    print("-------------------")
                    time.sleep(5)                
            # If no next token exists
            else:
                if result_count is not None and result_count > 0:
                    print("-------------------")
                    print("Start Date: ", start_list[i])
                    append_to_csv(json_response, keyword+"_data.csv")
                    count += result_count
                    total_tweets += result_count
                    print("Total # of Tweets added: ", total_tweets)
                    print("-------------------")
                    time.sleep(5)
                
                #Since this is the final request, turn flag to false to move to the next time period.
                flag = False
                next_token = None
            time.sleep(5)
        print("Total number of results: ", total_tweets)



In [14]:
def append_to_csv(json_response, fileName):
    

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):
            if('place_id' in tweet['geo']):
                geo = tweet['geo']['place_id']
            else:
                print(tweet['geo'])
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        #source = tweet['source']

        if ('source' in tweet):   
            source= tweet['source']
        else:
            source = " "

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 



In [15]:
keywords =["with guilt","saddled", "the guilt" ,"compunction", "misgiving", "prick", "qualm", "scruple","fault","liability", "rap", 
           "responsibility","chagrin", "sorrow","bloodguilt", "bloodguiltiness","apology", "excuses", "hand-wringing", "mea culpa","contriteness",
           "contrition", "penitence", "regret","embarrassment","anguish", "distress", "grief", "ruth", "sadness","remorsefulness", 
           "repentance","blame", "culpability"," remorse","rue", "self-reproach", "shame","#feelingguilty","I wish","#GUILT","#GUILTYthoughts", "#GUILTYconcience", "#mybad","#iconfess"]

In [ ]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
#keywords = ["#feelingguilty","I wish","#GUILT","#GUILTYthoughts", "#GUILTYconcience", "#mybad","#iconfess"]



start_list =    ['2011-01-01T00:00:00.000Z',
                 '2012-01-01T00:00:00.000Z',
                 '2013-01-01T00:00:00.000Z',
                 '2014-01-01T00:00:00.000Z',
                 '2015-01-01T00:00:00.000Z',
                 '2016-01-01T00:00:00.000Z',
                 '2017-01-01T00:00:00.000Z',
                 '2018-01-01T00:00:00.000Z',
                 '2019-01-01T00:00:00.000Z',
                 '2020-01-01T00:00:00.000Z',
                 '2021-01-01T00:00:00.000Z',]

end_list =      ['2011-12-31T00:00:00.000Z',
                 '2012-12-31T00:00:00.000Z',
                 '2013-12-31T00:00:00.000Z',
                 '2014-12-31T00:00:00.000Z',
                 '2015-12-31T00:00:00.000Z',
                 '2016-12-31T00:00:00.000Z',
                 '2017-12-31T00:00:00.000Z',
                 '2018-12-31T00:00:00.000Z',
                 '2019-12-31T00:00:00.000Z',
                 '2020-12-31T00:00:00.000Z',
                 '2021-12-31T00:00:00.000Z',]
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

for x in keywords:
    fetch_tweets(x,total_tweets,500, start_list, end_list)


In [17]:




import pandas as pd
import glob 
import os 
import sys
# setting the path for joining multiple files 
files = os.path.join(sys.path[0], "*.csv") 
# list of merged files returned 
files = glob.glob(files) 
print("Resultant CSV after joining all CSV files at a particular location..."); 
# joining files with concat and read_csv 
df = pd.concat(map(pd.read_csv, files), ignore_index=True) 
print(df)

df.to_csv('allguilttweets.csv', header=True, index=False, columns=list(df.axes[1]))


Resultant CSV after joining all CSV files at a particular location...
                 author id                 created_at geo  \
0                134693758  2011-12-30 23:59:53+00:00       
1                109984248  2011-12-30 23:59:38+00:00       
2                 25256735  2011-12-30 23:59:36+00:00       
3                206338082  2011-12-30 23:59:05+00:00       
4                 37929470  2011-12-30 23:58:57+00:00       
...                    ...                        ...  ..   
302558  715095151442599936  2021-12-20 15:13:37+00:00       
302559          1467345817  2021-12-20 14:58:57+00:00       
302560          1473346147  2021-12-20 14:48:22+00:00       
302561           588683095  2021-12-20 13:55:25+00:00       
302562           282575696  2021-12-20 13:38:09+00:00       

                         id lang like_count quote_count reply_count  \
0        152901751677255680   en          0           0           1   
1        152901691501592578   en          0           0

In [23]:
extension = 'csv' 
all_filenames = [i for i in glob.glob('./*.{}'.format(extension))]

appended_data = []
for infile in all_filenames:
    data = pd.read_csv(infile, engine='python')
    # store DataFrame in list
    appended_data.append(data)
# see pd.concat documentation for more info
appended_data = pd.concat(appended_data)
# write DataFrame to an excel sheet 
appended_data.to_csv( "combined_cleaned_raw_keywords_twitter.csv", index=False, encoding='utf-8-sig')


In [29]:
appended_data.drop_duplicates(inplace=True)

In [30]:
appended_data.shape

(433320, 11)